In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project/2023/KT_ETRI

/content/drive/MyDrive/Project/2023/KT_ETRI


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

from xgboost.sklearn import XGBRegressor

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### 전처리 마친 데이터 불러오기


In [ ]:
train_df = pd.read_csv('Q1_data/Q1_train_pre.csv')
test_df = pd.read_csv('Q1_data/Q1_test_pre.csv')
train_df.head()

,datetime,ru_id,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,...,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,2023-05-19 00:00:00,BaseStationA,0.0,9,100.0,5,100.0,2,100.0,7,...,306.0,23.6,6.4,17.0,11.0,2083.0,-92.8,11.2,3,1
1,2023-05-19 00:05:00,BaseStationA,8.3,12,100.0,9,100.0,0,0.0,15,...,9385.0,15.6,2.4,19.0,15.0,1936.0,-92.8,10.7,11,2
2,2023-05-19 00:10:00,BaseStationA,0.0,9,100.0,8,100.0,4,100.0,11,...,19534.0,17.5,9.9,28.0,23.0,1874.0,-92.7,10.7,5,3
3,2023-05-19 00:15:00,BaseStationA,0.0,11,100.0,10,100.0,0,0.0,8,...,2260.0,19.9,5.0,16.0,14.0,1815.0,-92.6,9.4,8,2
4,2023-05-19 00:20:00,BaseStationA,0.0,9,100.0,8,100.0,0,0.0,9,...,446.0,17.1,0.6,14.0,10.0,1784.0,-92.8,9.6,8,2


# 모델링

In [ ]:
data = train_df.iloc[:, 2:]
data.describe()

,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,endcmodbysgnbratio,connestabatt,...,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,...,1.374480e+05,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000
mean,2.325792,22.161079,97.319247,17.025872,96.439811,3.223641,66.551252,15.928016,94.734347,7.347149,...,4.980100e+04,17.410996,8.858492,87.892534,35.523020,3915.484292,-92.025809,10.989813,9.409981,3.718825
std,8.017694,22.515226,16.096032,18.095286,18.506481,5.963709,47.150096,18.553752,20.611533,13.722298,...,1.269952e+05,13.782209,11.145691,83.697974,37.782572,2152.033668,1.501092,1.736028,17.130399,2.781932
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-96.600000,0.000000,0.000000,0.000000
25%,0.000000,8.000000,100.000000,7.000000,100.000000,0.000000,0.000000,5.000000,100.000000,1.000000,...,2.582000e+03,13.500000,4.300000,34.000000,14.000000,1863.000000,-92.600000,10.200000,1.000000,2.000000
50%,0.000000,18.000000,100.000000,14.000000,100.000000,2.000000,100.000000,12.000000,100.000000,4.000000,...,1.862400e+04,15.700000,6.900000,62.000000,26.000000,3800.000000,-92.200000,11.200000,4.000000,3.000000
75%,1.400000,30.000000,100.000000,23.000000,100.000000,4.000000,100.000000,22.000000,100.000000,9.000000,...,5.978700e+04,18.500000,10.000000,119.000000,45.000000,5585.000000,-91.500000,12.000000,13.000000,5.000000
max,250.000000,823.000000,100.000000,725.000000,100.000000,348.000000,100.000000,664.000000,100.000000,419.000000,...,1.098665e+07,400.000000,300.000000,4587.000000,1487.000000,9590.000000,0.000000,15.000000,817.000000,74.000000


In [ ]:
X, y = data.iloc[:, :-1], data.iloc[:, [-1]]

# train : valid = 7 : 3 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, shuffle = False, random_state = 30)
print(X_train.shape, X_valid.shape)

(96213, 33) (41235, 33)


In [ ]:
X_train.head()

,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,endcmodbysgnbratio,connestabatt,...,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,0.0,9,100.0,5,100.0,2,100.0,7,100.0,1,...,203.0,306.0,23.6,6.4,17.0,11.0,2083.0,-92.8,11.2,3
1,8.3,12,100.0,9,100.0,0,0.0,15,100.0,1,...,1181.0,9385.0,15.6,2.4,19.0,15.0,1936.0,-92.8,10.7,11
2,0.0,9,100.0,8,100.0,4,100.0,11,100.0,10,...,586.0,19534.0,17.5,9.9,28.0,23.0,1874.0,-92.7,10.7,5
3,0.0,11,100.0,10,100.0,0,0.0,8,100.0,2,...,434.0,2260.0,19.9,5.0,16.0,14.0,1815.0,-92.6,9.4,8
4,0.0,9,100.0,8,100.0,0,0.0,9,100.0,0,...,226.0,446.0,17.1,0.6,14.0,10.0,1784.0,-92.8,9.6,8


In [ ]:
# scaling
ss = StandardScaler()

X_train_scale = ss.fit_transform(X_train)
X_valid_scale = ss.transform(X_valid)

### 튜닝 x

In [ ]:
xgboost = XGBRegressor(random_state = 1)

In [ ]:
xgboost.fit(X_train_scale, y_train,
            eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
            verbose = 50)

[0]	validation_0-mae:1.24517
[50]	validation_0-mae:0.53611
[99]	validation_0-mae:0.53603


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5227045942967453


### 튜닝 o

##### max_depth 튜닝

In [ ]:
xgboost2 = XGBRegressor(n_estimators = 1000, max_depth = 8,
                        random_state = 1)

In [ ]:
xgboost2.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 50)

[0]	validation_0-mae:1.25248
[50]	validation_0-mae:0.53964
[100]	validation_0-mae:0.54637
[105]	validation_0-mae:0.54737


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost2.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5104804454231076


max_depth = 8 : 0.5104804454231076

max_depth = 12 : 0.5154848139148106

##### learning_rate 튜닝

다른 파라미터들 튜닝한 다음에 다시

In [ ]:
xgboost3 = XGBRegressor(n_estimators = 1000, max_depth = 8, learning_rate = 0.01,
                        random_state = 1)

In [ ]:
xgboost3.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 50)

[0]	validation_0-mae:1.79935
[50]	validation_0-mae:1.06983
[100]	validation_0-mae:0.70755
[150]	validation_0-mae:0.55818
[200]	validation_0-mae:0.51243
[250]	validation_0-mae:0.50401
[300]	validation_0-mae:0.50610
[348]	validation_0-mae:0.51021


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost3.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5040005442346915


##### min_child_weight 튜닝

In [ ]:
params = {
    'min_child_weight' : range(1, 10, 2)
}
cv = 3

estimator = XGBRegressor(n_estimators = 1000, max_depth = 8,
                        random_state = 1)

gridsearch = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    cv = cv,
    n_jobs = -1,
    scoring = 'neg_mean_absolute_error'
)

In [ ]:
gridsearch.fit(X_train_scale, y_train,
               eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
               verbose = 50)
gridsearch.best_params_

[0]	validation_0-mae:1.25256
[50]	validation_0-mae:0.53551
[100]	validation_0-mae:0.53896
[105]	validation_0-mae:0.53912


{'min_child_weight': 3}

왜 계속 gridsearch해서 얻은 값보다 다른 값 넣었을 때 valid set에서 mae가 더 작게 나오지?

In [ ]:
xgboost4 = XGBRegressor(n_estimators = 1000, max_depth = 8, min_child_weight = 2,
                        random_state = 1)

In [ ]:
xgboost4.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 50)

[0]	validation_0-mae:1.25251
[50]	validation_0-mae:0.53890
[100]	validation_0-mae:0.54713
[105]	validation_0-mae:0.54744


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost4.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5063835728789324


##### gamma 튜닝

In [ ]:
params = {
    'gamma' : [0, 0.05, 0.1]
}
cv = 3

estimator = XGBRegressor(n_estimators = 1000, max_depth = 8, min_child_weight = 2,
                        random_state = 1)

gridsearch = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    cv = cv,
    n_jobs = -1,
    scoring = 'neg_mean_absolute_error'
)

In [ ]:
gridsearch.fit(X_train_scale, y_train,
               eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
               verbose = 50)
gridsearch.best_params_

[0]	validation_0-mae:1.25251
[50]	validation_0-mae:0.53890
[100]	validation_0-mae:0.54713
[106]	validation_0-mae:0.54757


{'gamma': 0}

gamma는 딱히 영향 없는 듯

In [ ]:
xgboost5 = XGBRegressor(n_estimators = 1000, max_depth = 8, min_child_weight = 2, gamma = 0.05,
                        random_state = 1)

In [ ]:
xgboost5.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 50)

[0]	validation_0-mae:1.25251
[50]	validation_0-mae:0.53438
[100]	validation_0-mae:0.53982
[106]	validation_0-mae:0.53950


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.05, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost5.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5063835728789324


##### 다시 learning_rate 튜닝

In [ ]:
xgboost6 = XGBRegressor(n_estimators = 1000, max_depth = 8, min_child_weight = 2, learning_rate = 0.005,
                        random_state = 1)

In [ ]:
xgboost6.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 50)

[0]	validation_0-mae:1.80884
[50]	validation_0-mae:1.38840
[100]	validation_0-mae:1.07635
[150]	validation_0-mae:0.85932
[200]	validation_0-mae:0.71091
[250]	validation_0-mae:0.61556
[300]	validation_0-mae:0.55921
[350]	validation_0-mae:0.52815
[400]	validation_0-mae:0.51250
[450]	validation_0-mae:0.50565
[500]	validation_0-mae:0.50363
[550]	validation_0-mae:0.50426
[600]	validation_0-mae:0.50573
[608]	validation_0-mae:0.50599


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [ ]:
y_pred = xgboost6.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5036259267363994


lr = 0.01 : 0.5037553992623462

lr = 0.05 : 0.5049462094538458

lr = 0.005 : 0.5036259267363994

lr = 0.001 : 0.5129252023397756

validation set 기준 0.5 아래로 안떨어지네......

기존 전처리 방식으로는 xgboost6 모델에서 개선하기 힘들듯

### 제출 파일 만들기

In [ ]:
def predict_df(model) :
  label_sample = pd.read_csv('Q1_data/Q1_label_sample.csv')

  StationB = test_df[test_df['ru_id'] == 'BaseStationB'].iloc[:, 2:]
  StationJ = test_df[test_df['ru_id'] == 'BaseStationJ'].iloc[:, 2:]

  B_scale = ss.transform(StationB)
  B_pred = model.predict(B_scale)

  J_scale = ss.transform(StationJ)
  J_pred = model.predict(J_scale)

  label_sample['BaseStationB'] = B_pred
  label_sample['BaseStationJ'] = J_pred

  return label_sample

In [ ]:
label_sample = predict_df(xgboost6)

In [ ]:
label_sample.to_csv('Q1_data/xgb_tuning.csv', index = False)